# Prediction Model for SWAPI Data

### Import training data

In [3]:
import pandas as pd

main_df = pd.read_csv('troop_movements.csv')
main_df

,timestamp,unit_id,unit_type,empire_or_resistance,location_x,location_y,destination_x,destination_y,homeworld
0,2024-02-06 16:45:34,1,tie_fighter,resistance,6,9,6,7,Chandrila
1,2024-02-06 16:45:33,2,stormtrooper,resistance,8,2,3,6,Ryloth
2,2024-02-06 16:45:32,3,tie_fighter,resistance,2,6,8,7,Rodia
3,2024-02-06 16:45:31,4,tie_fighter,resistance,5,3,2,9,Chandrila
4,2024-02-06 16:45:30,5,x-wing,empire,8,4,10,3,Aleen Minor
...,...,...,...,...,...,...,...,...,...
995,2024-02-06 16:28:59,996,unknown,resistance,10,10,2,6,Shili
996,2024-02-06 16:28:58,997,tie_silencer,resistance,9,10,9,9,Dorin
997,2024-02-06 16:28:57,998,tie_fighter,empire,5,5,9,10,Chandrila
998,2024-02-06 16:28:56,999,stormtrooper,empire,10,9,6,2,Skako


### Create grouped data showing counts of empire v resistance